In [26]:
import numpy as np
import torch
import torch.nn as nn
import sys

In [27]:
def wSum(X,W):
    h = torch.from_numpy(X)
    z = torch.matmul(W,h)
    return z

In [28]:
def activate(x):
    return 1/(1+torch.exp(-x))

In [29]:
def forwardStep(X,W_list):
    h = torch.from_numpy(X)
    for W in W_list:
        z = torch.matmul(W,h)
        h = activate(z)
    return h

In [30]:
def updateParams(W_list,dW_list,lr):
    with torch.no_grad():
        for i in range(len(W_list)):
            W_list[i] -= lr*dW_list[i]
    return W_list

In [31]:
def trainNN_sgd(X,y,W_list,loss_fn,lr=0.0001,nepochs=100):
    for epoch in range(nepochs):
        avgLoss = []
        for i in range(len(y)):
            Xin = X[i,:]
            yTrue = y[i]
            y_hat = forwardStep(Xin,W_list)
            loss = loss_fn(y_hat,torch.tensor(yTrue,dtype=torch.double))
            loss.backward()
            avgLoss.append(loss.item())
            sys.stdout.flush()
            dW_list = []
            for j in range(len(W_list)):
                dW_list.append(W_list[j].grad.data)
            W_list = updateParams(W_list,dW_list,lr)
            for j in range(len(W_list)):
                W_list[j].grad.data.zero_()
        print("Loss after epoch=%d: %f" %(epoch,np.mean(np.array(avgLoss))))
    return W_list

In [32]:
def trainNN_batch(X,y,W_list,loss_fn,lr=0.0001,nepochs=100):
    n = len(y) # Compute number
    for epoch in range(nepochs):
        loss = 0
        for i in range(n):
            Xin = X[i,:]
            yTrue = y[i]
            y_hat = forwardStep(Xin,W_list)
            # Compute Total Loss Then we update.
            loss += loss_fn(y_hat,torch.tensor(yTrue,dtype=torch.double))
        loss = loss/n  # Average loss of the epoch.
        loss.backward() 
        sys.stdout.flush()
        dW_list = []
        for j in range(len(W_list)):
            dW_list.append(W_list[j].grad.data)
        W_list = updateParams(W_list,dW_list,lr)
        for j in range(len(W_list)):
            W_list[j].grad.data.zero_()
        print("Loss after epoch=%d: %f" %(epoch,loss))
    return W_list

In [33]:
def trainNN_minibatch(X,y,W_list,loss_fn,lr=0.0001,nepochs=100,batchSize=16):
    n = len(y)
    numBatches = n//batchSize
    
    for epoch in range(nepochs):
        for batch in range(numBatches):
            X_batch = X[batch*batchSize:(batch+1)*batchSize,:]
            y_batch = y[batch*batchSize:(batch+1)*batchSize]
            loss = 0
            for i in range(batchSize):
                Xin = X_batch[i,:]
                yTrue = y_batch[i]
                y_hat = forwardStep(Xin,W_list)
                loss += loss_fn(y_hat,torch.tensor(yTrue,dtype=torch.double))
            loss = loss/batchSize
            loss.backward()
            sys.stdout.flush()
            dW_list = []
            for j in range(len(W_list)):
                dW_list.append(W_list[j].grad.data)
            W_list = updateParams(W_list,dW_list,lr)
            for j in range(len(W_list)):
                W_list[j].grad.data.zero_()
        print("Loss after epoch=%d: %f" %(epoch,loss/numBatches))
    return W_list

In [34]:
inputDim = 10
n = 1000
X = np.random.rand(n,inputDim)
y = np.random.randint(0,2,n)

W1 = torch.tensor(np.random.uniform(0,1,(2,inputDim)),requires_grad=True)
W2 = torch.tensor(np.random.uniform(0,1,(3,2)),requires_grad=True)
W3 = torch.tensor(np.random.uniform(0,1,3),requires_grad=True)

W_list = []
W_list.append(W1)
W_list.append(W2)
W_list.append(W3)

loss_fn = nn.BCELoss()
#W_list = trainNN_sgd(X,y,W_list,loss_fn,lr=0.0001,nepochs=100)
#W_list = trainNN_batch(X,y,W_list,loss_fn,lr=0.0001,nepochs=100)
W_list = trainNN_batch(X,y,W_list,loss_fn,lr=0.0001,nepochs=100)

Loss after epoch=0: 0.715217
Loss after epoch=1: 0.715216
Loss after epoch=2: 0.715214
Loss after epoch=3: 0.715212
Loss after epoch=4: 0.715211
Loss after epoch=5: 0.715209
Loss after epoch=6: 0.715208
Loss after epoch=7: 0.715206
Loss after epoch=8: 0.715205
Loss after epoch=9: 0.715203
Loss after epoch=10: 0.715201
Loss after epoch=11: 0.715200
Loss after epoch=12: 0.715198
Loss after epoch=13: 0.715197
Loss after epoch=14: 0.715195
Loss after epoch=15: 0.715194
Loss after epoch=16: 0.715192
Loss after epoch=17: 0.715190
Loss after epoch=18: 0.715189
Loss after epoch=19: 0.715187
Loss after epoch=20: 0.715186
Loss after epoch=21: 0.715184
Loss after epoch=22: 0.715183
Loss after epoch=23: 0.715181
Loss after epoch=24: 0.715179
Loss after epoch=25: 0.715178
Loss after epoch=26: 0.715176
Loss after epoch=27: 0.715175
Loss after epoch=28: 0.715173
Loss after epoch=29: 0.715172
Loss after epoch=30: 0.715170
Loss after epoch=31: 0.715168
Loss after epoch=32: 0.715167
Loss after epoch=33:

In [35]:
inputDim = 10
n = 1000
X = np.random.rand(n,inputDim)
y = np.random.randint(0,2,n)

In [36]:
X.shape

(1000, 10)

In [37]:
y.shape

(1000,)

In [38]:
np.unique(y)

array([0, 1])

In [39]:
W = torch.tensor(np.random.uniform(0,1,inputDim),requires_grad=True)

In [40]:
z = wSum(X[0,:],W)

In [41]:
print(z)

tensor(2.2336, dtype=torch.float64, grad_fn=<DotBackward0>)


In [42]:
inputDim = 10
n = 1000
X = np.random.rand(n,inputDim)
y = np.random.randint(0,2,n)

W1 = torch.tensor(np.random.uniform(0,1,(2,inputDim)),requires_grad=True)
W2 = torch.tensor(np.random.uniform(0,1,(3,2)),requires_grad=True)
W3 = torch.tensor(np.random.uniform(0,1,3),requires_grad=True)

W_list = []
W_list.append(W1)
W_list.append(W2)
W_list.append(W3)
z = forwardStep(X[0,:],W_list)
print(z)

tensor(0.5785, dtype=torch.float64, grad_fn=<MulBackward0>)


In [43]:
m = nn.Sigmoid()
loss_fun = nn.BCELoss()
lr = 0.0001
x = torch.randn(1)
y = torch.randint(0,2,(1,),dtype=torch.float)
w = torch.randn(1,requires_grad=True)

In [44]:
nIter = 100
for i in range(nIter):
    y_hat = m(w*x)
    loss = loss_fun(y_hat,y)
    loss.backward()
    dw = w.grad.data
    with torch.no_grad():
        w -= lr*dw
    w.grad.data.zero_()
    print(loss.item())

0.7737970948219299
0.7737958431243896
0.7737945318222046
0.77379310131073
0.7737917900085449
0.7737905383110046
0.77378910779953
0.773787796497345
0.7737863659858704
0.7737851142883301
0.773783802986145
0.77378249168396
0.7737810611724854
0.7737798094749451
0.77377849817276
0.7737770676612854
0.7737757563591003
0.7737745046615601
0.7737730741500854
0.7737717628479004
0.7737705111503601
0.7737690806388855
0.7737677693367004
0.7737664580345154
0.7737652063369751
0.7737637758255005
0.7737624645233154
0.7737610340118408
0.7737597823143005
0.7737584710121155
0.7737570405006409
0.7737559080123901
0.7737544775009155
0.7737531661987305
0.7737517356872559
0.7737504839897156
0.773749053478241
0.7737477421760559
0.7737464308738708
0.7737451791763306
0.773743748664856
0.7737424373626709
0.7737411260604858
0.773739755153656
0.773738443851471
0.7737370133399963
0.7737358808517456
0.7737345695495605
0.7737331390380859
0.7737318277359009
0.773730456829071
0.773729145526886
0.7737278342247009
0.7737265

In [45]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader

In [46]:
inputDim = 10
n = 1000
X = np.random.rand(n,inputDim)
y = np.random.randint(0,2,n)

tensor_x = torch.Tensor(X)
tensor_y = torch.Tensor(y)
Xy = TensorDataset(tensor_x,tensor_y)
Xy_loader = DataLoader(Xy,batch_size=16,shuffle=True,drop_last=True)

In [47]:
model = nn.Sequential(
    nn.Linear(inputDim,200),
    nn.ReLU(),
    #nn.BatchNorm1d(num_features=200),
    nn.Dropout(0.5),
    nn.Linear(200,100),
    nn.Tanh(),
    #nn.BatchNorm1d(num_features=100),
    nn.Linear(100,1),
    nn.Sigmoid()
)

In [48]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [49]:
loss_fn = nn.BCELoss()

In [50]:
nepochs = 100
for epoch in range(nepochs):
    for X,y in Xy_loader:
        batch_size = X.shape[0]
        y_hat = model(X.view(batch_size,-1))
        loss = loss_fn(y_hat,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(float(loss))

ValueError: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.

In [ ]:
with torch.no_grad():
    xt = torch.tensor(np.random.rand(1,inputDim))
    y2 = model(xt.float())
    print(y2.detach().numpy()[0][0])